In [12]:
# Importing some basic libraries

import numpy as np
import pandas as pd

# Importing the dataset

air = pd.read_csv('airline_df_nlped.csv')
air.head(20)

,Country,Airline,Review,Review_LS,Sentiment
0,China,Air China,los angeles beijing return. food low quality s...,los angeles beijing return . food quality staf...,0
1,China,Air China,round trip from hong kong to munich. the main ...,round trip hong kong munich . main reason fly ...,0
2,China,Air China,sydney beijing paris then rome beijing to sydn...,sydney beijing paris rome beijing sydney famil...,0
3,China,Air China,london to sydney return via beijing. a cheap f...,london sydney return beijing . cheap flight li...,0
4,China,Air China,beijing to shanghai. only one check in desk fo...,beijing shanghai . check desk standby passenge...,1
5,China,Air China,stockholm bangkok via beijing. return flight b...,stockholm bangkok beijing . return flight bang...,0
6,China,Air China,vancouver to hong kong via beijing. seating wa...,vancouver hong kong beijing . seating comforta...,0
7,China,Air China,vancouver to beijing return in economy. my veg...,vancouver beijing return economy . vegetarian ...,1
8,China,Air China,hong kong) bjs on a321. old looking plane but ...,hong kong ) bjs a321 . plane clean modern dome...,0
9,China,Air China,dus beijing manila return. 2.5 hours delay to ...,dus beijing manila return . 2.5 hour delay bei...,0


In [2]:
air['Review'][19]

'paris to beijing in business class on a 777 300 and return from shanghai to paris in business class on a340. seating on the 777 is outstanding with new seats large tv screens and a 2 2 2 configuration. on the a340 the configuration was the same but there were older seats smaller tv screens though i cannot complain about seating. on both legs staff were friendly and attentive. what is unsatisfying was the quality of the meals. i have seldom tasted such bland food and the presentation was horrible. before take off only orange juice and water were served. wines good selection of french wines were outstanding but must have been very expensive because there was no second serving nor were liquors offered at the end of the meal. another drawback was the total absence of reading materials in english or in french except a few china daily newspapers. a large proportion of the offered movies were oldies. if you are not interested in food if you bring your own reading materials air china is a goo

In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

In [4]:
tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
model = AutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")
model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")

In [5]:
tokens = tokenizer.encode(
    "paris to beijing in business class on a 777 300 and return from shanghai to paris in business class on a340. seating on the 777 is outstanding with new seats large tv screens and a 2 2 2 configuration. on the a340 the configuration was the same but there were older seats smaller tv screens though i cannot complain about seating. on both legs staff were friendly and attentive. what is unsatisfying was the quality of the meals. i have seldom tasted such bland food and the presentation was horrible. before take off only orange juice and water were served. wines good selection of french wines were outstanding but must have been very expensive because there was no second serving nor were liquors offered at the end of the meal. another drawback was the total absence of reading materials in english or in french except a few china daily newspapers. a large proportion of the offered movies were oldies. if you are not interested in food if you bring your own reading materials air china is a good option as it is cheap and not too many people fly it so you have plenty of space.",
    return_tensors="pt",
)

In [6]:
tokens

tensor([[  101, 10718, 10114, 20885, 10104, 12425, 12728, 10125,   143, 43630,
         11023, 10110, 14142, 10195, 23563, 10114, 10718, 10104, 12425, 12728,
         10125, 57212, 36722,   119, 16749, 10285, 10125, 10103, 43630, 10127,
         26056, 10171, 10246, 26471, 12166, 10827, 21496, 10107, 10110,   143,
           123,   123,   123, 50900,   119, 10125, 10103, 57212, 36722, 10103,
         50900, 10140, 10103, 11714, 10502, 10768, 10342, 18585, 26471, 21788,
         10827, 21496, 10107, 14325,   151, 25004, 85065, 44409, 10935, 16749,
         10285,   119, 10125, 11422, 42492, 15929, 10342, 35751, 10110, 54740,
         12894, 10598,   119, 11523, 10127, 15905, 13112, 45220, 38101, 10140,
         10103, 19468, 10108, 10103, 82714, 12339,   119,   151, 10574, 23789,
         17034, 44960, 10163, 11165, 16675, 15225, 10110, 10103, 33289, 10140,
         36129, 45795, 10301,   119, 11364, 11622, 11856, 10902, 17419, 70580,
         10110, 11917, 10342, 12567,   119, 28312, 1

In [7]:
tokenizer.decode(tokens[0])

'[CLS] paris to beijing in business class on a 777 300 and return from shanghai to paris in business class on a340. seating on the 777 is outstanding with new seats large tv screens and a 2 2 2 configuration. on the a340 the configuration was the same but there were older seats smaller tv screens though i cannot complain about seating. on both legs staff were friendly and attentive. what is unsatisfying was the quality of the meals. i have seldom tasted such bland food and the presentation was horrible. before take off only orange juice and water were served. wines good selection of french wines were outstanding but must have been very expensive because there was no second serving nor were liquors offered at the end of the meal. another drawback was the total absence of reading materials in english or in french except a few china daily newspapers. a large proportion of the offered movies were oldies. if you are not interested in food if you bring your own reading materials air china is

In [8]:
reults = model(tokens)
reults

SequenceClassifierOutput(loss=None, logits=tensor([[-0.4403,  0.8183,  0.8859,  0.3805, -1.4286]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [10]:
int(torch.argmax(reults.logits))+1

3